In [16]:
import pandas as pd
import time
from tqdm import tqdm
from functools import reduce
import os

In [11]:
input_dir = '../ppi_ml/results/elutions/elut_excavate/'
ext = 'mFDRpsm001.unique.norm.elut'
output_dir = '../ppi_ml/results/elutions/elut_ordered/'
outfile = output_dir+'excavate.unique.norm.elut.ordered'

In [12]:
files = [fn for fn in os.listdir(input_dir) if fn.endswith(ext)]
files.sort()

In [13]:
files[:10]

['euggr.sec_1.prot_count_mFDRpsm001.unique.norm.elut',
 'tryb2.sax_1.prot_count_mFDRpsm001.unique.norm.elut',
 'tryb2.sec_1.prot_count_mFDRpsm001.unique.norm.elut',
 'tryb2.sec_2.prot_count_mFDRpsm001.unique.norm.elut']

In [ ]:
def get_files(input_dir, ext):
    
    files = [fn for fn in os.listdir(input_dir) if fn.endswith(ext)]
    files.sort()
    
    return files

In [5]:
def read_csv_pgbar(csv_path, chunksize, dtype=object):
    
    rows = sum(1 for _ in open(csv_path, 'r')) - 1 # minus the header
    chunk_list = []
    with tqdm(total=rows, desc='Rows read: ') as bar:
        for chunk in pd.read_csv(csv_path, chunksize=chunksize,
                                 dtype=dtype):
            chunk_list.append(chunk)
            bar.update(len(chunk))
    df = pd.concat((f for f in chunk_list), axis=0)
    print('Done!')
    
    return df

In [ ]:
def parse_files(input_dir, files):
    
    df_list = []
    for i in range(len(files)):
        f = files[i]
        file_path = input_dir+f
        print(f'Parsing {f} (exp #{i}) ...')
        df = read_csv_pgbar(file_path, 100)
        df_list.append(df)
    
    return df_list

In [ ]:
def merge_files(df_list, outfile):
    
    print('Merging all experiments ...')
    df_merged = pd.concat(df_list).fillna(0, inplace=True)
    print(df_merged.head())
    print(f'Writing output to {outfile} ...')
    df_merged.to_csv(outfile, index=False)

In [14]:
df_list = []
for i in range(len(files)):
    f = files[i]
    file_path = input_dir+f
    print(f'Parsing {f} (exp #{i}) ...')
    df = read_csv_pgbar(file_path, 100)
    df_list.append(df)

Parsing euggr.sec_1.prot_count_mFDRpsm001.unique.norm.elut (exp #0) ...


Rows read: 100%|█████████████████████████████████████████████████████████| 15244/15244 [00:00<00:00, 54582.76it/s]


Done!
Parsing tryb2.sax_1.prot_count_mFDRpsm001.unique.norm.elut (exp #1) ...


Rows read: 100%|███████████████████████████████████████████████████████████| 6852/6852 [00:00<00:00, 25186.71it/s]


Done!
Parsing tryb2.sec_1.prot_count_mFDRpsm001.unique.norm.elut (exp #2) ...


Rows read: 100%|███████████████████████████████████████████████████████████| 6965/6965 [00:00<00:00, 10927.57it/s]


Done!
Parsing tryb2.sec_2.prot_count_mFDRpsm001.unique.norm.elut (exp #3) ...


Rows read: 100%|███████████████████████████████████████████████████████████| 6961/6961 [00:00<00:00, 10697.73it/s]

Done!


In [19]:
print('Merging all experiments ...')
#df_merged = pd.merge(df_list).fillna(0, inplace=True)
df_merged = reduce(lambda x, y: pd.merge(x, y, on='orthogroup', how='outer'), df_list)
df_merged

Merging all experiments ...


,orthogroup,Euglena_SEC_16a_01062017,Euglena_SEC_18a_01072017,Euglena_SEC_20a_01072017,Euglena_SEC_21a_01072017,Euglena_SEC_22a_01082017,Euglena_SEC_23a_01082017,Euglena_SEC_24a_01082017,Euglena_SEC_25a_01082017,Euglena_SEC_26a_01082017,...,PTSS4538SN19110,PTSS4538SN19111,PTSS4538SN19112,PTSS4538SN19113,PTSS4538SN19114,PTSS4538SN19115,PTSS4538SN19116,PTSS4538SN19117,PTSS4538SN19118,PTSS4538SN19119
0,EG_transcript_10,0,0,0,0,1,0.5,0.5,0.5,0.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EG_transcript_1000,0,0,1,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,EG_transcript_10000,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,EG_transcript_10004,0,0,0,0,1,1,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EG_transcript_10007,0.07142857142857142,0,0.14285714285714285,0.6428571428571429,0.5714285714285714,0.7857142857142857,0.7857142857142857,0.5714285714285714,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20357,Q57WI7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
20358,Q57Y32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
20359,Q584M3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,0,0,0,0,0,0,0
20360,Q586K7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [21]:
df_merged.isnull().sum().sum()

7845626

In [9]:
df_merged.to_csv(outfile, index=False)

AttributeError: 'NoneType' object has no attribute 'to_csv'